In [131]:
import relaqs.api.gates as gates
import numpy as np
import pandas as pd
from qutip.superoperator import liouvillian, spre, spost
from qutip import Qobj, tensor, Bloch, basis, sigmax, sigmay, sigmaz, expect
from qutip.operators import *
import matplotlib.pyplot as plt
from relaqs.api.utils import *
import scipy.linalg as la

In [74]:
I = gates.I().get_matrix()
X = gates.X().get_matrix()
Y = gates.Y().get_matrix()
Z = gates.Z().get_matrix()

In [196]:
steps_per_Haar = 1
num_haar = 1
final_time = 35.5556E-9 #in seconds
gamma_phase_max = 1.1675 * np.pi
gamma_magnitude_max = 1.8 * np.pi / final_time / steps_per_Haar
alpha_max = 0.05E9  # detuning of the control pulse in Hz

In [204]:
def compare_actions_U_target(action,u_target, u_initial):
    num_time_bins = 2 ** (steps_per_Haar - 1) # Haar number decides the number of time bins
    # Compute actual control parameters
    gamma_magnitude = gamma_magnitude_max/2 * (action[0]+1)
    gamma_phase = gamma_phase_max * action[1]
    alpha = alpha_max * action[2]

    H_array = []
    H_tot = []
    U_array = []

    # Construct the Hamiltonian
    psi_0 = np.array([[1], [0]], dtype=complex)  # |0⟩ state
    H = alpha*Z + gamma_magnitude*(np.cos(gamma_phase)*X + np.sin(gamma_phase)*Y)
    H_array.append(H)
    H_tot = []
    for ii, H_elem in enumerate(H_array):
        for jj in range(0, num_time_bins):
            Haar_num = num_haar - np.floor(ii / steps_per_Haar) # Haar_num: label which Haar wavelet, current_Haar_num: order in the array
            factor = (-1) ** np.floor(jj / (2 ** (Haar_num - 1)))
            if ii > 0:
                H_tot[jj] += factor * H_elem
            else:
                H_tot.append(factor * H_elem)

    U = u_initial.copy()
    for jj in range(0, num_time_bins):
        Ut = la.expm(-1j * final_time / num_time_bins * H_tot[jj])
        U = Ut @ U
    U_array.append(U)

    vector_target = np.matmul(u_target, np.array([[1], [0]]))  # Apply gate to |0⟩ = [1, 0]^T
    vector_initial = np.matmul(u_initial, np.array([[1], [0]]))
    vector_action = np.matmul(U, np.array([[1], [0]]))
    # q_obj = [qutip.Qobj(vector_initial),qutip.Qobj(vector_target), qutip.Qobj(vector_action)]  # Convert the resulting vector to a QuTiP quantum object
    q_obj = [qutip.Qobj(vector_initial),qutip.Qobj(vector_target)]  # Convert the resulting vector to a QuTiP quantum object
    bloch_sphere = Bloch()
    bloch_sphere.vector_color = ['m','y', 'g']  # magenta for clarity
    bloch_sphere.point_color = ['b']
    bloch_sphere.add_states(q_obj)

    psi_final = U @ psi_0
    rho = psi_final @ psi_final.conj().T

    x = np.real(np.trace(rho @ X))
    y = np.real(np.trace(rho @ Y))
    z = np.real(np.trace(rho @ Z))
    bloch_sphere.add_points(np.array([x,y,z]).T)

    bloch_sphere.show()

In [209]:
def unitary_to_superoperator(U):
    S1 = np.kron(U.conj(), U)
    S2 = np.kron(U, U.conj())
    S3 = np.kron(U, U.conj().T)
    S4 = np.array((spre(Qobj(U)) * spost(Qobj(U))).data.toarray())
    S5 = np.array((spre(Qobj(U)) * spost(Qobj(U.conj()))).data.toarray())
    return S1, S2, S3, S4, S5

In [199]:
def process_fidelity(S1, S2):
    d = int(np.sqrt(S1.shape[0]))  # Extract system dimension
    return np.abs(np.trace(S1.conj().T @ S2)) / d**2

In [210]:
data_path = '/Users/vishchaudhary/rl-repo/results/2025-03-12_20-52-02/RandomSU2/RandomSU2_env_data.csv'
df = pd.read_csv(data_path,header = 0)

fidelities = df.iloc[:, 0]
print(f'Overall mean of Inference: {fidelities.mean()}\n')
preprocessed_actions = df.iloc[:, 2].apply(preprocess_actions)
preprocessed_operator = df.iloc[:, 3].apply(preprocess_matrix_string)
preprocessed_target_operator = df.iloc[:, 4].apply(preprocess_matrix_string)
preprocessed_U_target = df.iloc[:, 5].apply(preprocess_matrix_string)
preprocessed_U_initial = df.iloc[:, 6].apply(preprocess_matrix_string)

actions_array = [np.array(eval(m)) for m in preprocessed_actions]
u_target_list = [np.array(eval(m)) for m in preprocessed_U_target]
u_target_list = np.array(u_target_list)

u_initial_list = [np.array(eval(m)) for m in preprocessed_U_initial]
u_initial_list = np.array(u_initial_list)

operator = [np.array(eval(m)) for m in preprocessed_operator]
operator = np.array(operator)

target_operator = [np.array(eval(m)) for m in preprocessed_target_operator]
target_operator = np.array(target_operator)

for i in range(10):
    if fidelities[i] > 0.99:
        print(fidelities[i])
        print('U_initial [Magenta],\tU_target [Yellow],\tCalculated Point from actions [Blue].')
        compare_actions_U_target(actions_array[i], u_target_list[i], u_initial_list[i])

0.9949318709060032
S1:
[[ 0.9078464 -9.16392978e-18j  0.08871169+2.75302642e-01j
   0.08871169-2.75302642e-01j  0.09215359-6.27803356e-19j]
 [ 0.2637422 -1.18749156e-01j -0.60864661-6.73598099e-01j
  -0.01023845-9.15830711e-02j -0.2637422 +1.18749156e-01j]
 [ 0.2637422 +1.18749156e-01j -0.01023845+9.15830711e-02j
  -0.60864661+6.73598099e-01j -0.2637422 -1.18749156e-01j]
 [ 0.09215359+6.27803356e-19j -0.08871169-2.75302642e-01j
  -0.08871169+2.75302642e-01j  0.9078464 +9.16392978e-18j]]
S2:
[[ 0.9078464 +9.16392978e-18j  0.08871169-2.75302642e-01j
   0.08871169+2.75302642e-01j  0.09215359+6.27803356e-19j]
 [ 0.2637422 +1.18749156e-01j -0.60864661+6.73598099e-01j
  -0.01023845+9.15830711e-02j -0.2637422 -1.18749156e-01j]
 [ 0.2637422 -1.18749156e-01j -0.01023845-9.15830711e-02j
  -0.60864661-6.73598099e-01j -0.2637422 +1.18749156e-01j]
 [ 0.09215359-6.27803356e-19j -0.08871169+2.75302642e-01j
  -0.08871169-2.75302642e-01j  0.9078464 -9.16392978e-18j]]
S3:
[[-0.60864661+0.6735981j  -0.26